In [1]:
from collections import namedtuple
import functools

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from forecast.games_dataset_features import *
from forecast.ml.module import ModuleExt


Dataset = namedtuple('Dataset', ['train_x', 'train_y', 'test_x', 'test_y'])
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [10]:
dataset_file = 'games_10_3'
x_origin = np.load('./server/forecast/dataset/{}_x.npy'.format(dataset_file))
y = np.load('./server/forecast/dataset/{}_y.npy'.format(dataset_file))

print(x_origin.shape)
print(y.shape)

# y2 is a target for win/lose only
y2 = y.copy()
y2[y2 == 1] = 0
y2[y2 == 2] = 1
y2[y2 == 3] = 1
print(y2.shape)


(6615, 107)
(6615,)
(6615,)


In [3]:
from math import floor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


def prepare_dataset(features, target, test_data_percentage=0.2, scaler=None, transform_func=None):
    x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=test_data_percentage)
    if scaler is not None:
        scaler.fit(x_train)  
        x_train = scaler.transform(x_train)  
        x_test = scaler.transform(x_test)
    if transform_func is not None:
        x_train = transform_func(x_train)
        x_test = transform_func(x_test)
    return Dataset(
        torch.from_numpy(x_train).float(),
        torch.from_numpy(y_train).long(),
        torch.from_numpy(x_test).float(),
        torch.from_numpy(y_test).long()
    )


def evaluate_model(model, data):
    predictions = model.predict(data.test_x)
    print("Train Accuracy: ", accuracy_score(data.train_y, model.predict(data.train_x)))
    print("Test Accuracy: ", accuracy_score(data.test_y, predictions))
    print("Confusion matrix\n", confusion_matrix(data.test_y, predictions))


def transform_game_to_conv1(data):
    return data[:, FEATURES_STATS_ONLY].reshape((data.shape[0], 6, FEATURE_STAT_COUNT))


def get_model_output(model, input_size):
    def conv_output(in_size, kernel, stride, padding, dilation):
        return int(floor(((in_size + 2*padding - dilation*(kernel - 1) - 1) / stride) + 1))

    out_channels = 0
    d0, d1 = 0, 0
    if len(input_size) == 1:
        d0, = input_size
    elif len(input_size) == 2:
        d0, d1 = input_size

    for m in model.modules():
        if isinstance(m, nn.Sequential) or isinstance(m, nn.ModuleList):
            continue
        if isinstance(m, nn.Conv1d):
            d0 = conv_output(d0, m.kernel_size[0], m.stride[0], m.padding[0], m.dilation[0])
            out_channels = m.out_channels
        elif isinstance(m, nn.Conv2d):
            d0 = conv_output(d0, m.kernel_size[0], m.stride[0], m.padding[0], m.dilation[0])
            d1 = conv_output(d1, m.kernel_size[1], m.stride[1], m.padding[1], m.dilation[1])
            out_channels = m.out_channels
        elif isinstance(m, nn.MaxPool1d):
            d0 = conv_output(d0, m.kernel_size, m.stride, m.padding, m.dilation)
        elif isinstance(m, nn.MaxPool2d):
            d0 = conv_output(d0, m.kernel_size, m.stride, m.padding, m.dilation)
            d1 = conv_output(d1, m.kernel_size, m.stride, m.padding, m.dilation)
    return (out_channels, d0, d1)[:len(input_size) + 1]


In [11]:
from sklearn.preprocessing import StandardScaler
from torch.optim import Adam, SGD, RMSprop
from torch.optim.lr_scheduler import StepLR, ExponentialLR

from forecast.ml.hparam_tuning import RandomSearchCV


class GameModelFC(ModuleExt):
    def __init__(self, num_classes=2, num_features=10, hidden_size=None, dropout=None):
        super(GameModelFC, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(num_features, hidden_size[0]),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=dropout),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=dropout),
            nn.Linear(hidden_size[2], hidden_size[3]),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=dropout),
            nn.Linear(hidden_size[3], hidden_size[4]),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=dropout),
            nn.Linear(hidden_size[4], hidden_size[5]),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=dropout),
            nn.Linear(hidden_size[5], hidden_size[6]),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=dropout),
            nn.Linear(hidden_size[6], hidden_size[7]),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=dropout),
            nn.Linear(hidden_size[7], hidden_size[8]),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=dropout),
            nn.Linear(hidden_size[8], hidden_size[9]),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_size[9], hidden_size[10]),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size[10], num_classes),
        )

    def forward(self, x):
        return self.classifier(x)


scaler = StandardScaler()
dataset = prepare_dataset(x_origin, y2, scaler=scaler)

params = {
    'mod_num_classes': [2],
    'mod_num_features': [FEATURE_COUNT],
    'mod_hidden_size': [
        [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100],
        [100, 200, 200, 200, 300, 300, 200, 200, 200, 100, 100],
        [100, 200, 200, 200, 300, 300, 200, 200, 200, 100, 50]
    ],
    'mod_dropout': [0.5],
    'mod_init_weights': ['he'],
    'fit_epochs': [90],
    'opt_cls': [Adam, SGD, RMSprop],
    'opt_lr': [1e-03, 1e-04, 1e-05],
    'ann_cls': [StepLR],
    'ann_step_size': [30]
}

rscv = RandomSearchCV(GameModelFC, params, try_count=15, cv=5, verbose=1)
rscv.fit(dataset.train_x, dataset.train_y)
print('\n\n')
print(rscv.best_params)
print(rscv.best_score)

evaluate_model(rscv.best_model, dataset)
net = rscv.best_model



Fitting for each of 15 candidates



Parameters: {'opt_lr': 0.001, 'opt_cls': <class 'torch.optim.sgd.SGD'>, 'mod_num_features': 107, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_hidden_size': [100, 200, 200, 200, 300, 300, 200, 200, 200, 100, 100], 'mod_dropout': 0.5, 'fit_epochs': 90, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5311320754716982, time: 134.4786570072174



Parameters: {'opt_lr': 1e-05, 'opt_cls': <class 'torch.optim.adam.Adam'>, 'mod_num_features': 107, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_hidden_size': [100, 200, 200, 200, 300, 300, 200, 200, 200, 100, 100], 'mod_dropout': 0.5, 'fit_epochs': 90, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5274102079395085, time: 248.69393348693848



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.sgd.SGD'>, 'mod_num_features': 107, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_hidden_size': [100, 200, 200, 200, 300, 300, 200, 200, 200, 100, 50], 'mod_dropout': 0.5, 'fit_epochs': 90, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5444234404536862, time: 148.38032937049866



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.rmsprop.RMSprop'>, 'mod_num_features': 107, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_hidden_size': [100, 200, 200, 200, 300, 300, 200, 200, 200, 100, 50], 'mod_dropout': 0.5, 'fit_epochs': 90, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5434782608695652, time: 173.80008268356323



...

Parameters: {'opt_lr': 0.001, 'opt_cls': <class 'torch.optim.rmsprop.RMSprop'>, 'mod_num_features': 107, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_hidden_size': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 'mod_dropout': 0.5, 'fit_epochs': 90, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5113421550094518, time: 101.20760178565979



Parameters: {'opt_lr': 1e-05, 'opt_cls': <class 'torch.optim.rmsprop.RMSprop'>, 'mod_num_features': 107, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_hidden_size': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 'mod_dropout': 0.5, 'fit_epochs': 90, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5500945179584121, time: 88.29166293144226



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.sgd.SGD'>, 'mod_num_features': 107, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_hidden_size': [100, 200, 200, 200, 300, 300, 200, 200, 200, 100, 100], 'mod_dropout': 0.5, 'fit_epochs': 90, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5415879017013232, time: 134.341078042984



Parameters: {'opt_lr': 1e-05, 'opt_cls': <class 'torch.optim.rmsprop.RMSprop'>, 'mod_num_features': 107, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_hidden_size': [100, 200, 200, 200, 300, 300, 200, 200, 200, 100, 50], 'mod_dropout': 0.5, 'fit_epochs': 90, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5179584120982986, time: 192.78613901138306



{'opt_lr': 1e-05, 'opt_cls': <class 'torch.optim.rmsprop.RMSprop'>, 'mod_num_features': 107, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_hidden_size': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 'mod_dropout': 0.5, 'fit_epochs': 90, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
0.5500945179584121


Train Accuracy:  0.6781934996220711
Test Accuracy:  0.5185185185185185
Confusion matrix
 [[482 204]
 [433 204]]


In [5]:
ConvLayer = namedtuple('ConvLayer', ['c_in', 'c_out', 'c_kern', 'c_pad', 'mp_kern'])


class GameModel(ModuleExt):
    def __init__(self, num_classes=2, conv_layer_params=None, classifier_params=None):
        super(GameModel, self).__init__()
        hidden_size, dropout = classifier_params
        self.features = nn.Sequential(
            nn.Conv1d(conv_layer_params[0].c_in, conv_layer_params[0].c_out,
                      kernel_size=conv_layer_params[0].c_kern, padding=conv_layer_params[0].c_pad),
            nn.BatchNorm1d(conv_layer_params[0].c_out),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=conv_layer_params[0].mp_kern),

            nn.Conv1d(conv_layer_params[1].c_in, conv_layer_params[1].c_out,
                      kernel_size=conv_layer_params[1].c_kern, padding=conv_layer_params[1].c_pad),
            nn.BatchNorm1d(conv_layer_params[1].c_out),
            nn.ReLU(inplace=True),
            # nn.MaxPool1d(kernel_size=conv_layer_params[1].mp_kern),

            nn.Conv1d(conv_layer_params[2].c_in, conv_layer_params[2].c_out,
                      kernel_size=conv_layer_params[1].c_kern, padding=conv_layer_params[2].c_pad),
            nn.BatchNorm1d(conv_layer_params[2].c_out),
            nn.ReLU(inplace=True),
            # nn.MaxPool1d(kernel_size=conv_layer_params[2].mp_kern),

            nn.Conv1d(conv_layer_params[3].c_in, conv_layer_params[3].c_out,
                      kernel_size=conv_layer_params[3].c_kern, padding=conv_layer_params[3].c_pad),
            nn.BatchNorm1d(conv_layer_params[3].c_out),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=conv_layer_params[3].mp_kern),
        )
        l1_output = get_model_output(self.features, (FEATURE_STAT_COUNT,))
        flat_out = functools.reduce(lambda x, y: x*y, l1_output, 1)
        self.classifier = nn.Sequential(
            # nn.Dropout(p=dropout),
            nn.Linear(flat_out, hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, num_classes),
        )
        
    def forward(self, x):
        out = self.features(x)
        out = out.reshape(out.size(0), -1)
        out = self.classifier(out)
        return out


In [12]:
from sklearn.preprocessing import StandardScaler
from torch.optim import Adam, SGD, RMSprop
from torch.optim.lr_scheduler import StepLR, ExponentialLR

from forecast.ml.hparam_tuning import RandomSearchCV

scaler = StandardScaler()
dataset = prepare_dataset(x_origin, y2, scaler=scaler, transform_func=transform_game_to_conv1)

params = {
    'mod_num_classes': [2],
    'mod_conv_layer_params': [
        [ConvLayer(6, 20, 1, 0, 2), ConvLayer(20, 50, 1, 0, 2), ConvLayer(50, 80, 1, 0, 2), ConvLayer(80, 100, 1, 0, 2)],
        [ConvLayer(6, 20, 2, 1, 2), ConvLayer(20, 50, 2, 1, 2), ConvLayer(50, 50, 2, 1, 2), ConvLayer(50, 80, 2, 1, 2)]
    ],
    'mod_classifier_params': [(100, 0.5), (120, 0.5)],
    'mod_init_weights': ['he'],
    'fit_epochs': [80, 120],
    'opt_cls': [Adam, RMSprop],
    'opt_lr': [0.0001],
    'ann_cls': [StepLR],
    'ann_step_size': [30]
}

rscv = RandomSearchCV(GameModel, params, try_count=8, cv=5, verbose=1)
rscv.fit(dataset.train_x, dataset.train_y)
print('\n\n')
print(rscv.best_params)
print(rscv.best_score)

evaluate_model(rscv.best_model, dataset)
net = rscv.best_model



Fitting for each of 8 candidates



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.adam.Adam'>, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_conv_layer_params': [ConvLayer(c_in=6, c_out=20, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=20, c_out=50, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=50, c_out=50, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=50, c_out=80, c_kern=2, c_pad=1, mp_kern=2)], 'mod_classifier_params': (100, 0.5), 'fit_epochs': 80, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5118035882908404, time: 230.2893660068512



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.rmsprop.RMSprop'>, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_conv_layer_params': [ConvLayer(c_in=6, c_out=20, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=20, c_out=50, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=50, c_out=50, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=50, c_out=80, c_kern=2, c_pad=1, mp_kern=2)], 'mod_classifier_params': (120, 0.5), 'fit_epochs': 80, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5136921624173749, time: 204.582772731781



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.rmsprop.RMSprop'>, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_conv_layer_params': [ConvLayer(c_in=6, c_out=20, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=20, c_out=50, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=50, c_out=80, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=80, c_out=100, c_kern=1, c_pad=0, mp_kern=2)], 'mod_classifier_params': (100, 0.5), 'fit_epochs': 120, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5179584120982986, time: 273.0029032230377



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.adam.Adam'>, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_conv_layer_params': [ConvLayer(c_in=6, c_out=20, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=20, c_out=50, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=50, c_out=50, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=50, c_out=80, c_kern=2, c_pad=1, mp_kern=2)], 'mod_classifier_params': (100, 0.5), 'fit_epochs': 120, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.500945179584121, time: 310.53158950805664



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.rmsprop.RMSprop'>, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_conv_layer_params': [ConvLayer(c_in=6, c_out=20, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=20, c_out=50, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=50, c_out=50, c_kern=2, c_pad=1, mp_kern=2), ConvLayer(c_in=50, c_out=80, c_kern=2, c_pad=1, mp_kern=2)], 'mod_classifier_params': (120, 0.5), 'fit_epochs': 120, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5359168241965974, time: 309.2411103248596



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.adam.Adam'>, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_conv_layer_params': [ConvLayer(c_in=6, c_out=20, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=20, c_out=50, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=50, c_out=80, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=80, c_out=100, c_kern=1, c_pad=0, mp_kern=2)], 'mod_classifier_params': (120, 0.5), 'fit_epochs': 80, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5231350330500472, time: 187.00966000556946



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.adam.Adam'>, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_conv_layer_params': [ConvLayer(c_in=6, c_out=20, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=20, c_out=50, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=50, c_out=80, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=80, c_out=100, c_kern=1, c_pad=0, mp_kern=2)], 'mod_classifier_params': (100, 0.5), 'fit_epochs': 120, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5316336166194523, time: 306.3104827404022



Parameters: {'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.adam.Adam'>, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_conv_layer_params': [ConvLayer(c_in=6, c_out=20, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=20, c_out=50, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=50, c_out=80, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=80, c_out=100, c_kern=1, c_pad=0, mp_kern=2)], 'mod_classifier_params': (100, 0.5), 'fit_epochs': 80, 'ann_step_size': 30, 'ann_cls': <class 'torch.optim.lr_scheduler.StepLR'>}
Result: score: 0.5378071833648393, time: 212.24958062171936



{'opt_lr': 0.0001, 'opt_cls': <class 'torch.optim.adam.Adam'>, 'mod_num_classes': 2, 'mod_init_weights': 'he', 'mod_conv_layer_params': [ConvLayer(c_in=6, c_out=20, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=20, c_out=50, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=50, c_out=80, c_kern=1, c_pad=0, mp_kern=2), ConvLayer(c_in=80, c_out=100, c_kern=1, c_pad=0, mp_kern=2)], 'mod_classifier_params': (100, 0.5), 'f

Train Accuracy:  0.8252078609221466
Test Accuracy:  0.5011337868480725
Confusion matrix
 [[416 301]
 [359 247]]


In [ ]:
from sklearn.preprocessing import StandardScaler
from torch.optim import Adam, SGD, RMSprop
from torch.optim.lr_scheduler import StepLR, ExponentialLR

from forecast.ml.hparam_tuning import RandomSearchCV

scaler = StandardScaler()
dataset = prepare_dataset(x_origin, y2, scaler=scaler, transform_func=transform_game_to_conv1)

net = GameModel(conv_layer_params=(6, 1, 0, 1), classifier_params=(12, 0.5))
net.init_weights(mode='he')

optimizer = Adam(net.parameters(), lr=0.0001)
annealing = StepLR(optimizer, step_size=30)

net.fit(dataset.train_x, dataset.train_y, optimizer, annealing=annealing, epochs=80, validation_split=0.3)

evaluate_model(net, dataset)


In [38]:
import matplotlib.pyplot as plt

x_axis = range(0, net.history.shape[0])

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.set_ylim(0, 1.5)
ax1.plot(x_axis, net.history[:, ModuleExt.HISTORY_TRAIN_LOSS], 'ro-', label='Train loss')
ax1.plot(x_axis, net.history[:, ModuleExt.HISTORY_VAL_LOSS], 'bo-', label='Validation loss')
ax1.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left', ncol=2, mode="expand", borderaxespad=0.)

ax2.set_ylim(0, 1)
ax2.plot(x_axis, net.history[:, ModuleExt.HISTORY_TRAIN_ACC], 'ro-', label='Train acc')
ax2.plot(x_axis, net.history[:, ModuleExt.HISTORY_VAL_ACC], 'bo-', label='Validation acc')
ax2.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left', ncol=2, mode="expand", borderaxespad=0.)

plt.show()


(6, 6)
